In [ ]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

In [ ]:
df_selecao = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name = 'selecoes',index_col=0)
df_jogos  = pd.read_excel('DadosCopaDoMundoQatar2022.xlsx', sheet_name='jogos')

In [ ]:
f = df_selecao["PontosRankingFIFA"]
a, b = min(f), max(f) 
fa, fb = 0.15, 1 
b1 = (fb - fa)/(b-a) 
b0 = fb - b*b1
forca = b0 + b1*f

In [ ]:
def media_equipes(selecao1, selecao2):
    forca_casa = forca[f'{selecao1}']
    forca_fora = forca[f'{selecao2}']
    expectativa_de_gols = 2.72
    lambda_casa = expectativa_de_gols*forca_casa/(forca_casa + forca_fora)
    lambda_fora = expectativa_de_gols*forca_fora/(forca_fora + forca_fora)
    return [lambda_casa, lambda_fora]

def distribuicao(media):
    probs = []
    for i in range(7):
        probs.append(poisson.pmf(i,media))
    probs.append(1-sum(probs))
    return pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6', '7+'])

def qtd_gols(casa,fora):
    lambda_casa, lambda_fora = media_equipes(selecao1=casa,selecao2=fora)
    gols_casa = np.random.poisson(lam=lambda_casa,size=1)
    gols_fora = np.random.poisson(lam=lambda_fora,size=1)
    saldo_time_casa = gols_casa - gols_fora
    saldo_time_fora = -saldo_time_casa
    placar = f'{saldo_time_casa}x{saldo_time_fora}'
    return [gols_casa, gols_fora, saldo_time_casa, saldo_time_fora, placar]


def probabilidade(selecao1,selecao2):
    lambda_casa, lambda_fora = media_equipes(selecao1=selecao1, selecao2=selecao2)
    dist1, dist2 = distribuicao(media = lambda_casa), distribuicao(media= lambda_fora)
    matriz = np.outer(dist1, dist2)
    vitoria = np.tril(matriz).sum()-np.trace(matriz)    #Soma a triangulo inferior
    derrota = np.triu(matriz).sum()-np.trace(matriz)    #Soma a triangulo superior
    empate = 1 - (vitoria + derrota)
    probs = np.around([vitoria, empate , derrota], 3)
    
    probsp = [f'{100*i:.1f}%' for i in probs]

    nomes = ['0', '1', '2', '3', '4', '5', '6', '7+']
    matriz = pd.DataFrame(matriz, columns = nomes, index = nomes)
    matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
    matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns]) 

    output = {'seleção1': selecao1, 'seleção2': selecao2, 
             'f1': forca[selecao1], 'f2': forca[selecao2], 
             'media1': lambda_casa, 'media2': lambda_fora, 
             'probabilidades': probsp, 'matriz': matriz}
    
    return output

def Resultado(gols1, gols2):
    if gols1 > gols2:
        res = 'V'
    if gols1 < gols2:
        res = 'D' 
    if gols1 == gols2:
        res = 'E'       
    return res

def Pontos(gols1, gols2):
    rst = Resultado(gols1, gols2)
    if rst == 'V':
        pontos1, pontos2 = 3, 0
    if rst == 'E':
        pontos1, pontos2 = 1, 1
    if rst == 'D':
        pontos1, pontos2 = 0, 3
    return pontos1, pontos2, rst

def Jogo(selecao1, selecao2):
    l1, l2 = media_equipes(selecao1, selecao2)
    gols1 = int(np.random.poisson(lam = l1, size = 1))
    gols2 = int(np.random.poisson(lam = l2, size = 1))
    saldo1 = gols1 - gols2
    saldo2 = -saldo1
    pontos1, pontos2, result = Pontos(gols1, gols2)
    placar = '{}x{}'.format(gols1, gols2)
    return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, result, placar]

In [ ]:
df_jogos['Vitoria'] = None
df_jogos['Empate'] = None
df_jogos['Derrota'] = None

for i in range(len(df_jogos)):
    s1 = df_jogos['seleção1'][i]
    s2 = df_jogos['seleção2'][i]
    v,e,d = probabilidade(selecao1=s1, selecao2=s2)['probabilidades']
    df_jogos.at[i,'Vitoria'] = v
    df_jogos.at[i,'Empate'] = e
    df_jogos.at[i,'Derrota'] = d

# Fase de Grupos

In [ ]:
def JogosGrupo(dados, grupo):
    
    times = list(dados.loc[dados['Grupo']==grupo].index)

    time1, time2, time3, time4 = times

    pt1, pt2, pt3, pt4 = 0, 0, 0, 0
    gp1, gp2, gp3, gp4 = 0, 0, 0, 0
    sg1, sg2, sg3, sg4 = 0, 0, 0, 0

    jogo1 = Jogo(time1, time2)
    jogo2 = Jogo(time3, time4)

    jogo3 = Jogo(time1, time3)
    jogo4 = Jogo(time2, time4)

    jogo5 = Jogo(time1, time4)
    jogo6 = Jogo(time2, time3)

    gp1, gp2, sg1, sg2, pt1, pt2 = gp1 + jogo1[0], gp2 + jogo1[1], sg1 + jogo1[2], sg2 + jogo1[3], pt1 + jogo1[4], pt2 + jogo1[5]
    gp3, gp4, sg3, sg4, pt3, pt4 = gp3 + jogo2[0], gp4 + jogo2[1], sg3 + jogo2[2], sg4 + jogo2[3], pt3 + jogo2[4], pt4 + jogo2[5]
    gp1, gp3, sg1, sg3, pt1, pt3 = gp1 + jogo3[0], gp3 + jogo3[1], sg1 + jogo3[2], sg3 + jogo3[3], pt1 + jogo3[4], pt3 + jogo3[5]
    gp2, gp4, sg2, sg4, pt2, pt4 = gp2 + jogo4[0], gp4 + jogo4[1], sg2 + jogo4[2], sg4 + jogo4[3], pt2 + jogo4[4], pt4 + jogo4[5]
    gp1, gp4, sg1, sg4, pt1, pt4 = gp1 + jogo5[0], gp4 + jogo5[1], sg1 + jogo5[2], sg4 + jogo5[3], pt1 + jogo5[4], pt4 + jogo5[5]
    gp2, gp3, sg2, sg3, pt2, pt3 = gp2 + jogo6[0], gp3 + jogo6[1], sg2 + jogo6[2], sg3 + jogo6[3], pt2 + jogo6[4], pt3 + jogo6[5]

    partidas = [ time1 + ' x ' + time2, 
                 time3 + ' x ' + time4,
                 time1 + ' x ' + time3, 
                 time2 + ' x ' + time4,
                 time1 + ' x ' + time4,
                 time2 + ' x ' + time3 ]

    resultados = [ jogo1[6], jogo2[6], jogo3[6], jogo4[6], jogo5[6], jogo6[6] ]
    placares = [ jogo1[-1], jogo2[-1], jogo3[-1], jogo4[-1], jogo5[-1], jogo6[-1] ] 
    cols = ['Pontos', 'Saldo de Gols', 'Gols Pró']
    tab = pd.DataFrame([[pt1, pt2, pt3, pt4], [sg1, sg2, sg3, sg4], [gp1, gp2, gp3, gp4]], index = cols, columns = times).transpose()
    
    tab = tab.sort_values(['Pontos', 'Saldo de Gols', 'Gols Pró'], ascending = False)
    tab['Posição'] = [1, 2, 3, 4]

    jogos = pd.DataFrame([partidas, placares, resultados]).transpose()
    jogos.columns = ['Partida', 'Placar', 'Resultado']

    return [tab, jogos]

In [ ]:
def JogoMM(selecao1, selecao2):
    r = Jogo(selecao1=selecao1,selecao2=selecao2)
    if r[6] == 'V':
        return selecao1
    elif r[6] == 'D':
        return selecao2
    else:
        return random.sample([selecao1, selecao2], 1)[0]

In [ ]:
def SimulaCopa(dados):

    # Data Frame que será usado para guardar as informações
    cols = ['1st', '2nd', '3rd', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']
    n = dados.shape[0]
    m = len(cols)
    aux = np.array(np.zeros(n*m).reshape(n, m))
    info = pd.DataFrame(aux, columns = cols, index = dados.index) 
    info = info.astype(int)

    top16 = []
    for i in ['A','B','C','D','E','F','G','H']:
        resultados = JogosGrupo(dados=dados,grupo=i)[0]
        top16+=resultados.index[:2].to_list()
        anomes = resultados.index.to_list()
        info.at[anomes[0], '1st'] = 1
        info.at[anomes[1], '2nd'] = 1
        info.at[anomes[2], '3rd'] = 1
        info.at[anomes[3], '4th'] = 1
    
    # Oitavas de finais
    qf1 = JogoMM(top16[0], top16[3])   #A1 x B2
    qf2 = JogoMM(top16[2], top16[1])   #B1 x A2
    qf3 = JogoMM(top16[4], top16[7])   #C1 x D2
    qf4 = JogoMM(top16[6], top16[5])   #D1 x C2
    qf5 = JogoMM(top16[8], top16[11])  #E1 x F2
    qf6 = JogoMM(top16[10], top16[9])  #F1 x E2
    qf7 = JogoMM(top16[12], top16[15]) #G1 x H2
    qf8 = JogoMM(top16[14], top16[13]) #H1 x G2

    # Quartas de finais
    top8 = [qf1, qf2, qf3, qf4, qf5, qf6, qf7, qf8]
    sf1 = JogoMM(qf1, qf3)
    sf2 = JogoMM(qf2, qf4) 
    sf3 = JogoMM(qf5, qf7) 
    sf4 = JogoMM(qf6, qf8) 

    # Semi-finais
    top4 = [sf1, sf2, sf3, sf4]
    f1 = JogoMM(sf1, sf3) 
    f2 = JogoMM(sf2, sf4) 
    
    # Final
    top2 = [f1, f2]
    top1 = JogoMM(f1,f2) 

    # Informações
    info.at[top16, 'Oitavas'] = 1
    info.at[top8, 'Quartas'] = 1
    info.at[top4, 'Semis'] = 1
    info.at[top2, 'Final'] = 1
    info.at[top1, 'Campeão'] = 1

    return info

In [ ]:
# Data Frame que será usado para guardar as informações
cols = ['1st', '2nd', '3rd', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']
n = df_selecao.shape[0]
m = len(cols)
aux = np.array(np.zeros(n*m).reshape(n, m))
info = pd.DataFrame(aux, columns = cols, index = df_selecao.index) 
info = info.astype(int)

top16 = []
for i in ['A','B','C','D','E','F','G','H']:
    resultados = JogosGrupo(dados=df_selecao,grupo=i)[0]
    top16+=resultados.index[:2].to_list()
    anomes = resultados.index.to_list()
    info.at[anomes[0], '1st'] = 1
    info.at[anomes[1], '2nd'] = 1
    info.at[anomes[2], '3rd'] = 1
    info.at[anomes[3], '4th'] = 1

# Oitavas de finais
qf1 = JogoMM(top16[0], top16[3])   #A1 x B2
qf2 = JogoMM(top16[2], top16[1])   #B1 x A2
qf3 = JogoMM(top16[4], top16[7])   #C1 x D2
qf4 = JogoMM(top16[6], top16[5])   #D1 x C2
qf5 = JogoMM(top16[8], top16[11])  #E1 x F2
qf6 = JogoMM(top16[10], top16[9])  #F1 x E2
qf7 = JogoMM(top16[12], top16[15]) #G1 x H2
qf8 = JogoMM(top16[14], top16[13]) #H1 x G2

# Quartas de finais
top8 = [qf1, qf2, qf3, qf4, qf5, qf6, qf7, qf8]
sf1 = JogoMM(qf1, qf3)
sf2 = JogoMM(qf2, qf4) 
sf3 = JogoMM(qf5, qf7) 
sf4 = JogoMM(qf6, qf8) 

# Semi-finais
top4 = [sf1, sf2, sf3, sf4]
f1 = JogoMM(sf1, sf3) 
f2 = JogoMM(sf2, sf4) 

# Final
top2 = [f1, f2]
top1 = JogoMM(f1,f2)

info.at[top16, 'Oitavas'] = 1